## Checks that the event codes are consistent with conditions for Attention Shift

This script cross-checks for consistency in `_eventstemp4.tsv` after
all corrections have been made notebook.

### Checking for forbidden codes
       Codes 1 and 2 can appear anywhere
       Codes 3 through 6 should appear only in the focus condition.
       Codes 7 through 14 should appear only in the shift condition.
       Codes 199, 201, 202, and 255 are not related to condition.

In [1]:
import os
import datetime
from hed.tools import BidsTabularDictionary, get_file_list, get_new_dataframe, HedLogger

# Variables to set for the specific dataset
bids_root_path = '/XXX/AttentionShiftWorking'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'run')
skip_cols = ['onset', 'duration', 'sample']
log_name = 'attention_shift_07_final_consistency_check_log'

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Construct the event file dictionary
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_eventstemp4", exclude_dirs=exclude_dirs)
bids_dict = BidsTabularDictionary("Event files", bids_files, entities=entities)

In [2]:
for key, file, rowcount, columns in bids_dict.iter_tsv_info():
    df_bids = get_new_dataframe(file.file_path)

    focus_cond_mask = df_bids['cond_code'].map(str).isin(['1', '2'])
    shift_cond_mask = df_bids['cond_code'].map(str).isin(['3'])
    focus_event_mask = df_bids['event_code'].map(str).isin(['3', '4', '5', '6'])
    shift_event_mask = df_bids['event_code'].map(str).isin(['7', '8', '9', '10', '11', '12', '13', '14'])
    bad_focus = sum(focus_cond_mask & shift_event_mask)
    if bad_focus:
        indices = df_bids.index[focus_cond_mask & shift_event_mask]
        logger.add(key, f"{key} has {bad_focus} shift event codes in a focus condition")
        logger.add(key, f"{str(list(indices))}")
    bad_shift = sum(shift_cond_mask & focus_event_mask)
    if bad_shift:
        indices = df_bids.index[shift_cond_mask & focus_event_mask]
        logger.add(key, f"{key} has {bad_shift} focus event codes in a shift condition")
        logger.add(key, f"{str(list(indices))}")
    bad_cond_mask = df_bids['cond_code'].map(str).isin(['0'])
    if sum(bad_cond_mask):
        logger.add(key, f"{key} has {sum(bad_cond_mask)} cond_code values of 0")
        logger.add(key, f"{str(list(df_bids.index[bad_cond_mask]))}")
    unknown_mask = df_bids['event_code'].map(str).isin(['255'])
    if sum(unknown_mask):
        logger.add(key, f"{key} has {sum(unknown_mask)} event_code values of 255")
        logger.add(key, f"{str(list(df_bids.index[unknown_mask]))}")
    auditory_shift_mask = df_bids['event_code'].map(str).isin(['7', '10', '11',  '14'])
    visual_shift_mask = df_bids['event_code'].map(str).isin(['8', '9', '12', '13'])
    visual_mask = df_bids['focus_modality'].map(str).isin(['visual', 'n/a'])
    auditory_mask = df_bids['focus_modality'].map(str).isin(['auditory', 'n/a'])
    bad_visual = shift_cond_mask & auditory_mask & visual_shift_mask
    if sum(bad_visual):
        logger.add(key, f"{key} has {sum(bad_visual)} visual event_code values in auditory focus modality")
        logger.add(key, f"{str(list(df_bids.index[bad_visual]))}")
    bad_auditory = shift_cond_mask & visual_mask & auditory_shift_mask
    if sum(bad_auditory):
        logger.add(key, f"{key} has {sum(bad_auditory)} auditory event_code values in visual focus modality")
        logger.add(key, f"{str(list(df_bids.index[bad_auditory]))}")

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
attention_shift_07_final_consistency_check_log: Level None
sub-005_run-01:
	[ sub-005_run-01 has 6 event_code values of 255]
	[ [266, 267, 268, 269, 270, 271]]


ERROR Summary:
attention_shift_07_final_consistency_check_log: Level ERROR
sub-005_run-01:
